# Data Combination

### Sam Timmins

In this notebook I will collect all the lap, weather, telemetry, etc. data and combine it into a single dataframe.

In [1]:
import pandas as pd
import fastf1
import numpy as np
from datetime import datetime

fastf1.set_log_level('WARNING')

In [2]:
def merge_weather_laps(session, track):
     # remove the in and out laps
    orig = session.laps[~session.laps['PitOutTime'].notna()]
    orig = orig[~orig['PitInTime'].notna()]
    
    # remove unneeded columns
    orig.drop(columns=['Time', 'DriverNumber', 'Stint', 'PitOutTime', 'PitInTime', 'SpeedST', 'IsPersonalBest', 
                 'LapStartTime', 'LapStartDate', 'Position', 'Deleted', 'DeletedReason', 'FastF1Generated', 
                 'IsAccurate'], inplace=True)
    
    # remove rows with null values
    orig.dropna(inplace=True)
    orig = orig[orig['TrackStatus'] == '1']
    orig['Track'] = track
    
    # merge the weather and lap data on the closest time of Sector 1
    return pd.merge_asof(orig.sort_values('Sector1SessionTime'), session.weather_data, left_on='Sector1SessionTime', right_on='Time')

In [3]:
years = [2022, 2023]
sessions = []

for year in years:
    schedule = fastf1.get_event_schedule(year) # get the schedule for each year
    
    for index, row in schedule.iterrows():
        if row['EventFormat'] != 'testing' and row['EventDate'] < datetime.now():
            race = fastf1.get_session(year, row['RoundNumber'], 'R')
            race.load(laps=True, weather=True, telemetry=True, messages=False)
            sessions.append(merge_weather_laps(race, row['Location']))   

req         WARNING 	

DEFAULT CACHE ENABLED!
	Cache directory: C:\Users\Sam\AppData\Local\Temp\fastf1.
	Size: 3.58 GB
core        WARNING 	No lap data for driver 22
core        WARNING 	No lap data for driver 47
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate.
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate.


In [4]:
orig = pd.concat(sessions)
orig.reset_index(drop=True, inplace=True)
orig

,Driver,LapTime,LapNumber,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,...,TrackStatus,Track,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,LEC,0 days 00:01:37.853000,2.0,0 days 00:00:31.582000,0 days 00:00:42.159000,0 days 00:00:24.112000,0 days 01:04:45.838000,0 days 01:05:27.997000,0 days 01:05:52.109000,226.0,...,1,Sakhir,0 days 01:04:03.474000,23.9,26.0,1010.4,False,29.0,13,0.3
1,SAI,0 days 00:01:38.649000,2.0,0 days 00:00:31.818000,0 days 00:00:42.486000,0 days 00:00:24.345000,0 days 01:04:47.941000,0 days 01:05:30.427000,0 days 01:05:54.772000,228.0,...,1,Sakhir,0 days 01:04:03.474000,23.9,26.0,1010.4,False,29.0,13,0.3
2,HAM,0 days 00:01:39.002000,2.0,0 days 00:00:31.844000,0 days 00:00:42.722000,0 days 00:00:24.436000,0 days 01:04:48.538000,0 days 01:05:31.260000,0 days 01:05:55.696000,230.0,...,1,Sakhir,0 days 01:04:03.474000,23.9,26.0,1010.4,False,29.0,13,0.3
3,MAG,0 days 00:01:38.993000,2.0,0 days 00:00:31.544000,0 days 00:00:42.940000,0 days 00:00:24.509000,0 days 01:04:49.047000,0 days 01:05:31.987000,0 days 01:05:56.496000,227.0,...,1,Sakhir,0 days 01:04:03.474000,23.9,26.0,1010.4,False,29.0,13,0.3
4,GAS,0 days 00:01:40.169000,2.0,0 days 00:00:32.180000,0 days 00:00:43.326000,0 days 00:00:24.663000,0 days 01:04:51.600000,0 days 01:05:34.926000,0 days 01:05:59.589000,227.0,...,1,Sakhir,0 days 01:04:03.474000,23.9,26.0,1010.4,False,29.0,13,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24779,PIA,0 days 00:01:24.503000,70.0,0 days 00:00:30.446000,0 days 00:00:30.086000,0 days 00:00:23.971000,0 days 02:40:21.521000,0 days 02:40:51.607000,0 days 02:41:15.578000,282.0,...,1,Budapest,0 days 02:39:51.961000,29.4,32.0,987.9,False,45.7,154,1.2
24780,LEC,0 days 00:01:24.451000,70.0,0 days 00:00:30.309000,0 days 00:00:30.144000,0 days 00:00:23.998000,0 days 02:40:24.164000,0 days 02:40:54.308000,0 days 02:41:18.306000,279.0,...,1,Budapest,0 days 02:39:51.961000,29.4,32.0,987.9,False,45.7,154,1.2
24781,RUS,0 days 00:01:24.254000,70.0,0 days 00:00:29.959000,0 days 00:00:30.447000,0 days 00:00:23.848000,0 days 02:40:24.525000,0 days 02:40:54.972000,0 days 02:41:18.820000,278.0,...,1,Budapest,0 days 02:39:51.961000,29.4,32.0,987.9,False,45.7,154,1.2
24782,ALO,0 days 00:01:24.181000,70.0,0 days 00:00:30.337000,0 days 00:00:29.940000,0 days 00:00:23.904000,0 days 02:40:34.868000,0 days 02:41:04.808000,0 days 02:41:28.712000,284.0,...,1,Budapest,0 days 02:39:51.961000,29.4,32.0,987.9,False,45.7,154,1.2


In [5]:
orig.drop(columns=['Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime', 'Time', 'TrackStatus'], inplace=True)
times = ['Sector1Time', 'Sector2Time', 'Sector3Time']
for time in times:
    orig[time] = orig[time].dt.total_seconds()

In [6]:
orig

,Driver,LapTime,LapNumber,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,Compound,...,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,LEC,0 days 00:01:37.853000,2.0,31.582,42.159,24.112,226.0,250.0,273.0,SOFT,...,True,Ferrari,Sakhir,23.9,26.0,1010.4,False,29.0,13,0.3
1,SAI,0 days 00:01:38.649000,2.0,31.818,42.486,24.345,228.0,250.0,275.0,SOFT,...,False,Ferrari,Sakhir,23.9,26.0,1010.4,False,29.0,13,0.3
2,HAM,0 days 00:01:39.002000,2.0,31.844,42.722,24.436,230.0,254.0,274.0,SOFT,...,False,Mercedes,Sakhir,23.9,26.0,1010.4,False,29.0,13,0.3
3,MAG,0 days 00:01:38.993000,2.0,31.544,42.940,24.509,227.0,251.0,278.0,SOFT,...,True,Haas F1 Team,Sakhir,23.9,26.0,1010.4,False,29.0,13,0.3
4,GAS,0 days 00:01:40.169000,2.0,32.180,43.326,24.663,227.0,242.0,274.0,SOFT,...,False,AlphaTauri,Sakhir,23.9,26.0,1010.4,False,29.0,13,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24779,PIA,0 days 00:01:24.503000,70.0,30.446,30.086,23.971,282.0,240.0,247.0,MEDIUM,...,False,McLaren,Budapest,29.4,32.0,987.9,False,45.7,154,1.2
24780,LEC,0 days 00:01:24.451000,70.0,30.309,30.144,23.998,279.0,238.0,245.0,HARD,...,False,Ferrari,Budapest,29.4,32.0,987.9,False,45.7,154,1.2
24781,RUS,0 days 00:01:24.254000,70.0,29.959,30.447,23.848,278.0,240.0,251.0,MEDIUM,...,True,Mercedes,Budapest,29.4,32.0,987.9,False,45.7,154,1.2
24782,ALO,0 days 00:01:24.181000,70.0,30.337,29.940,23.904,284.0,241.0,244.0,HARD,...,True,Aston Martin,Budapest,29.4,32.0,987.9,False,45.7,154,1.2


In [7]:
orig['FreshTyre'] = orig['FreshTyre'].astype(int)
orig['Rainfall'] = orig['Rainfall'].astype(int)
orig

,Driver,LapTime,LapNumber,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,Compound,...,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,LEC,0 days 00:01:37.853000,2.0,31.582,42.159,24.112,226.0,250.0,273.0,SOFT,...,1,Ferrari,Sakhir,23.9,26.0,1010.4,0,29.0,13,0.3
1,SAI,0 days 00:01:38.649000,2.0,31.818,42.486,24.345,228.0,250.0,275.0,SOFT,...,0,Ferrari,Sakhir,23.9,26.0,1010.4,0,29.0,13,0.3
2,HAM,0 days 00:01:39.002000,2.0,31.844,42.722,24.436,230.0,254.0,274.0,SOFT,...,0,Mercedes,Sakhir,23.9,26.0,1010.4,0,29.0,13,0.3
3,MAG,0 days 00:01:38.993000,2.0,31.544,42.940,24.509,227.0,251.0,278.0,SOFT,...,1,Haas F1 Team,Sakhir,23.9,26.0,1010.4,0,29.0,13,0.3
4,GAS,0 days 00:01:40.169000,2.0,32.180,43.326,24.663,227.0,242.0,274.0,SOFT,...,0,AlphaTauri,Sakhir,23.9,26.0,1010.4,0,29.0,13,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24779,PIA,0 days 00:01:24.503000,70.0,30.446,30.086,23.971,282.0,240.0,247.0,MEDIUM,...,0,McLaren,Budapest,29.4,32.0,987.9,0,45.7,154,1.2
24780,LEC,0 days 00:01:24.451000,70.0,30.309,30.144,23.998,279.0,238.0,245.0,HARD,...,0,Ferrari,Budapest,29.4,32.0,987.9,0,45.7,154,1.2
24781,RUS,0 days 00:01:24.254000,70.0,29.959,30.447,23.848,278.0,240.0,251.0,MEDIUM,...,1,Mercedes,Budapest,29.4,32.0,987.9,0,45.7,154,1.2
24782,ALO,0 days 00:01:24.181000,70.0,30.337,29.940,23.904,284.0,241.0,244.0,HARD,...,1,Aston Martin,Budapest,29.4,32.0,987.9,0,45.7,154,1.2


In [8]:
from sklearn.preprocessing import LabelEncoder

encoders = dict() # save each encoder in dictionary
categorical_features = ['Driver', 'Compound', 'Team', 'Track']
for col in categorical_features:
    orig[col] = orig[col].str.strip()
    encoders[col] = LabelEncoder() # save the encoder
    orig[col] = encoders[col].fit_transform(orig[col])
    
orig['Track'].value_counts()

2     2079
17    2006
11    1936
3     1814
12    1648
10    1501
1     1435
8     1323
14    1316
15    1193
9     1115
5     1070
4      791
20     788
19     728
21     723
0      697
6      682
13     607
16     547
7      519
18     266
Name: Track, dtype: int64

In [9]:
orig

,Driver,LapTime,LapNumber,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,Compound,...,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,8,0 days 00:01:37.853000,2.0,31.582,42.159,24.112,226.0,250.0,273.0,3,...,1,4,14,23.9,26.0,1010.4,0,29.0,13,0.3
1,17,0 days 00:01:38.649000,2.0,31.818,42.486,24.345,228.0,250.0,275.0,3,...,0,4,14,23.9,26.0,1010.4,0,29.0,13,0.3
2,5,0 days 00:01:39.002000,2.0,31.844,42.722,24.436,230.0,254.0,274.0,3,...,0,7,14,23.9,26.0,1010.4,0,29.0,13,0.3
3,9,0 days 00:01:38.993000,2.0,31.544,42.940,24.509,227.0,251.0,278.0,3,...,1,5,14,23.9,26.0,1010.4,0,29.0,13,0.3
4,4,0 days 00:01:40.169000,2.0,32.180,43.326,24.663,227.0,242.0,274.0,3,...,0,1,14,23.9,26.0,1010.4,0,29.0,13,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24779,14,0 days 00:01:24.503000,70.0,30.446,30.086,23.971,282.0,240.0,247.0,2,...,0,6,3,29.4,32.0,987.9,0,45.7,154,1.2
24780,8,0 days 00:01:24.451000,70.0,30.309,30.144,23.998,279.0,238.0,245.0,0,...,0,4,3,29.4,32.0,987.9,0,45.7,154,1.2
24781,16,0 days 00:01:24.254000,70.0,29.959,30.447,23.848,278.0,240.0,251.0,2,...,1,7,3,29.4,32.0,987.9,0,45.7,154,1.2
24782,1,0 days 00:01:24.181000,70.0,30.337,29.940,23.904,284.0,241.0,244.0,0,...,1,3,3,29.4,32.0,987.9,0,45.7,154,1.2


In [10]:
import pandas as pd

threshold = 1

# columns = ['Sector1Time', 'Sector2Time', 'Sector3Time']
columns = ['LapTime']
filtered_df = pd.DataFrame()
grouped = orig.groupby('Track')

for track, group in grouped:
    for column_name in columns:
        data = group[column_name]
        mean = data.mean()
        std = data.std()
        
        z_scores = (data - mean) / std
        filtered_rows = group[abs(z_scores) <= threshold]
        filtered_df = pd.concat([filtered_df, filtered_rows])

orig = filtered_df.drop_duplicates()

In [11]:
orig

,Driver,LapTime,LapNumber,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,Compound,...,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
12343,21,0 days 00:01:41.696000,2.0,27.804,40.703,33.189,205.0,179.0,199.0,2,...,1,8,0,28.9,52.0,991.5,0,34.6,92,3.9
12344,5,0 days 00:01:42.325000,2.0,28.114,41.003,33.208,202.0,179.0,199.0,2,...,1,7,0,28.9,52.0,991.5,0,34.6,92,3.9
12345,19,0 days 00:01:43.289000,2.0,28.123,41.422,33.744,197.0,177.0,198.0,2,...,0,3,0,28.9,52.0,991.5,0,34.6,92,3.9
12346,16,0 days 00:01:43.183000,2.0,28.195,41.229,33.759,204.0,183.0,200.0,2,...,1,7,0,28.9,52.0,991.5,0,34.6,92,3.9
12347,22,0 days 00:01:43.490000,2.0,28.242,41.327,33.921,206.0,178.0,197.0,2,...,0,3,0,28.9,52.0,991.5,0,34.6,92,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10946,0,0 days 00:01:17.233000,72.0,26.016,27.359,23.858,285.0,250.0,259.0,3,...,0,9,21,22.7,75.0,1017.2,0,33.6,187,1.1
10947,10,0 days 00:01:16.422000,72.0,25.847,27.441,23.134,284.0,268.0,296.0,3,...,0,5,21,22.7,75.0,1017.2,0,33.6,187,1.1
10948,9,0 days 00:01:16.825000,72.0,26.393,27.338,23.094,279.0,260.0,295.0,3,...,0,5,21,22.7,75.0,1017.2,0,33.6,187,1.1
10949,23,0 days 00:01:16.875000,72.0,26.415,27.282,23.178,285.0,276.0,312.0,3,...,0,0,21,22.7,75.0,1017.2,0,33.6,187,1.1


In [12]:
orig.drop(columns=['LapTime'], inplace=True)

In [13]:
orig['Track'].value_counts()

11    1534
17    1370
3     1334
15    1178
2     1141
12    1103
8     1084
10    1010
1      938
14     888
9      777
5      708
4      567
20     522
19     508
21     486
6      475
0      463
13     428
16     361
7      324
18     227
Name: Track, dtype: int64

In [14]:
orig['NextSectorTime'] = 0

In [15]:
dfs = []
for driver in orig.Driver.unique():
    for track in orig.Track.unique():
        temp = orig[(orig['Driver'] == driver) & (orig['Track'] == track)].sort_values('LapNumber').reset_index(drop=True)
        new_rows = []
        num_rows = len(temp) - 1
        for index, row in temp.iterrows():
            if index < num_rows:
                row = temp.loc[index].copy()
                row['LapNumber'] = row['LapNumber'] + 1
                row['NextSectorTime'] = temp.loc[index + 1]['Sector1Time']
                new_rows.append(row)
                
                row = temp.loc[index].copy()
                row['LapNumber'] = row['LapNumber'] + 1
                row['NextSectorTime'] = temp.loc[index + 1]['Sector2Time']
                row['Sector1Time'] = row['Sector2Time']
                row['Sector2Time'] = row['Sector3Time']
                row['Sector3Time'] = temp.loc[index + 1]['Sector1Time']
                new_rows.append(row)
                
                row = temp.loc[index].copy()
                row['LapNumber'] = row['LapNumber'] + 1
                row['NextSectorTime'] = temp.loc[index + 1]['Sector3Time']
                row['Sector1Time'] = row['Sector3Time']
                row['Sector2Time'] = temp.loc[index + 1]['Sector1Time']
                row['Sector3Time'] = temp.loc[index + 1]['Sector2Time']
                new_rows.append(row)
        
        dfs.append(pd.DataFrame(new_rows))

In [16]:
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df

,Driver,LapNumber,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,Compound,TyreLife,...,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,NextSectorTime
0,21.0,3.0,27.804,40.703,33.189,205.0,179.0,199.0,2.0,2.0,...,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,27.910
1,21.0,3.0,40.703,33.189,27.910,205.0,179.0,199.0,2.0,2.0,...,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,40.636
2,21.0,3.0,33.189,27.910,40.636,205.0,179.0,199.0,2.0,2.0,...,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,33.347
3,21.0,4.0,27.910,40.636,33.347,202.0,180.0,197.0,2.0,3.0,...,8.0,0.0,28.9,50.0,991.7,0.0,34.6,112.0,4.3,28.030
4,21.0,4.0,40.636,33.347,28.030,202.0,180.0,197.0,2.0,3.0,...,8.0,0.0,28.9,50.0,991.7,0.0,34.6,112.0,4.3,40.805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50872,3.0,69.0,31.591,21.384,17.560,299.0,230.0,273.0,2.0,31.0,...,1.0,17.0,22.8,53.0,934.0,0.0,31.3,59.0,0.8,31.624
50873,3.0,69.0,21.384,17.560,31.624,299.0,230.0,273.0,2.0,31.0,...,1.0,17.0,22.8,53.0,934.0,0.0,31.3,59.0,0.8,21.217
50874,3.0,70.0,17.560,31.624,21.217,298.0,231.0,273.0,2.0,32.0,...,1.0,17.0,22.7,53.0,934.0,0.0,31.5,27.0,0.8,17.679
50875,3.0,70.0,31.624,21.217,17.679,298.0,231.0,273.0,2.0,32.0,...,1.0,17.0,22.7,53.0,934.0,0.0,31.5,27.0,0.8,31.726


In [17]:
df.to_csv('data.csv', index=False)

In [18]:
dfs = []
for driver in orig.Driver.unique():
    for track in orig.Track.unique():
        temp = orig[(orig['Driver'] == driver) & (orig['Track'] == track)].sort_values('LapNumber').reset_index(drop=True)
        rows = []
        num_rows = len(temp) - 1
        for index, row in temp.iterrows():
            row = temp.loc[index].copy()
            row['NextSectorTime'] = row['Sector2Time']
            row.drop(columns=['Sector2Time', 'Sector3Time'], inplace=True)
            rows.append(row)
            
            row = temp.loc[index].copy()
            row['Sector1Time'] = row['Sector2Time']
            row['NextSectorTime'] = row['Sector3Time']
            row.drop(columns=['Sector2Time', 'Sector3Time'], inplace=True)
            rows.append(row)
            
            if index < num_rows:
                row = temp.loc[index].copy()
                row['LapNumber'] = row['LapNumber'] + 1
                row['Sector1Time'] = row['Sector3Time']
                row['NextSectorTime'] = temp.loc[index + 1]['Sector1Time']
                row.drop(columns=['Sector2Time', 'Sector3Time'], inplace=True)
                rows.append(row)
                
        dfs.append(pd.DataFrame(rows))                  

In [19]:
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df.drop(columns=['Sector2Time', 'Sector3Time'], inplace=True)
df.rename(columns={'Sector1Time':'SectorTime'}, inplace=True)
df

,Driver,LapNumber,SectorTime,SpeedI1,SpeedI2,SpeedFL,Compound,TyreLife,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,NextSectorTime
0,21.0,2.0,27.804,205.0,179.0,199.0,2.0,2.0,1.0,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,40.703
1,21.0,2.0,40.703,205.0,179.0,199.0,2.0,2.0,1.0,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,33.189
2,21.0,3.0,33.189,205.0,179.0,199.0,2.0,2.0,1.0,8.0,0.0,28.9,52.0,991.5,0.0,34.6,92.0,3.9,27.910
3,21.0,3.0,27.910,202.0,180.0,197.0,2.0,3.0,1.0,8.0,0.0,28.9,50.0,991.7,0.0,34.6,112.0,4.3,40.636
4,21.0,3.0,40.636,202.0,180.0,197.0,2.0,3.0,1.0,8.0,0.0,28.9,50.0,991.7,0.0,34.6,112.0,4.3,33.347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51806,3.0,69.0,17.560,298.0,231.0,273.0,2.0,32.0,1.0,1.0,17.0,22.7,53.0,934.0,0.0,31.5,27.0,0.8,31.624
51807,3.0,69.0,31.624,298.0,231.0,273.0,2.0,32.0,1.0,1.0,17.0,22.7,53.0,934.0,0.0,31.5,27.0,0.8,21.217
51808,3.0,70.0,21.217,298.0,231.0,273.0,2.0,32.0,1.0,1.0,17.0,22.7,53.0,934.0,0.0,31.5,27.0,0.8,17.679
51809,3.0,70.0,17.679,295.0,231.0,273.0,2.0,33.0,1.0,1.0,17.0,22.7,52.0,934.0,0.0,31.1,155.0,0.6,31.726


In [20]:
df.to_csv('single_sector_data.csv', index=False)

In [21]:
df.describe()

,Driver,LapNumber,SectorTime,SpeedI1,SpeedI2,SpeedFL,Compound,TyreLife,FreshTyre,Team,Track,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,NextSectorTime
count,51811.000000,51811.000000,51811.000000,51811.00000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000,51811.000000
mean,11.446507,30.763660,29.480604,252.81778,245.882361,269.013047,1.179325,15.672598,0.766652,4.398641,9.931848,23.987628,52.009014,986.505130,0.023219,35.157974,175.509718,1.907904,29.477278
std,7.078748,17.179698,7.150469,46.47063,40.220001,28.710603,1.152533,10.384107,0.422966,2.860902,5.935695,4.557758,18.145219,51.171662,0.150600,7.993575,102.569178,1.277572,7.154454
min,0.000000,2.000000,17.072000,95.00000,141.000000,120.000000,0.000000,1.000000,0.000000,0.000000,0.000000,13.000000,5.000000,779.500000,0.000000,15.600000,0.000000,0.000000,17.072000
25%,5.000000,17.000000,24.156000,213.00000,220.000000,247.000000,0.000000,8.000000,1.000000,2.000000,4.000000,21.300000,37.000000,988.000000,0.000000,29.900000,100.000000,1.000000,24.143000
50%,11.000000,30.000000,29.714000,269.00000,251.000000,274.000000,2.000000,13.000000,1.000000,4.000000,10.000000,24.000000,54.000000,1003.200000,0.000000,33.400000,164.000000,1.600000,29.707000
75%,17.000000,43.000000,33.237500,285.00000,277.000000,287.000000,2.000000,21.000000,1.000000,7.000000,15.000000,27.100000,64.000000,1013.200000,0.000000,40.700000,277.000000,2.400000,33.266000
max,23.000000,78.000000,52.920000,347.00000,339.000000,355.000000,4.000000,64.000000,1.000000,9.000000,21.000000,37.200000,91.000000,1018.500000,1.000000,57.500000,359.000000,8.600000,52.920000
